# ワークスペースへの接続

In [4]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [5]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="XXXXXXXXXXXXXXXXXXXXXXXXX",
    resource_group_name="XXXXXXXXXXXXXXXXX",
    workspace_name="XXXXXXXXXXXXXXXXXX",
)

# テストデータ準備（パターン２）

#### File SharesへのDataStore経由アクセス準備 : DataStoreの設定

In [ ]:
from azure.ai.ml.entities import AzureFileDatastore
from azure.ai.ml.entities._credentials import (
     AccountKeyConfiguration,
     CertificateConfiguration,
     NoneCredentialConfiguration,
     SasTokenConfiguration,
     ServicePrincipalConfiguration,
 )
from azure.ai.ml import MLClient

store = AzureFileDatastore(
    name="file_sas_example",
    description="Datastore pointing to a file share using sas token.",
    account_name="xxxxxxxxxxxxxxxxxxxxx",
    file_share_name="xxxxxxxxxxxxxxxxxxx",
    credentials=SasTokenConfiguration(
        sas_token="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
    ),
)
ml_client.create_or_update(store)

#### データアクセスと加工

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import UserIdentityConfiguration
from azure.ai.ml import Input

gpu_compute_target = "gpu-cluster-rainbow"
custom_env_name = "keras-env"
web_path = "azureml://datastores/file_sas_example/paths/<your path>"

job_env = ml_client.environments.get(name=custom_env_name, version=str(len(list(ml_client.environments.list(name=custom_env_name)))))
job = command(
    inputs=dict(
        data_folder=Input(type="uri_folder", path=web_path),
    ), 
    compute=gpu_compute_target,
    environment=f"{job_env.name}:{job_env.version}",
    code="./src/",
    command="python keras_image_preprocessing.py --data-folder ${{inputs.data_folder}}",
    experiment_name="xxxxx",
    display_name="xxxxx",
)

ml_client.jobs.create_or_update(job)